## Load data

In [1]:
import pandas as pd
import numpy as np

In [154]:
ks = pd.read_csv('../../../datasets/ks-projects-201801.csv')
ks.dropna(inplace=True)

In [155]:
# Now we have outcome=1 if successful, otherwise outcome=0
ks = ks.query('state != "live"')
ks = ks.assign(outcome=(ks['state'] == 'successful').astype(int))

ks.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,outcome
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95,0
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00,0
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00,0
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00,0
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00,0


In [156]:
# Drop some columns
ks.drop(['ID', 'name', 'category', 'currency', 'deadline', 'goal', 'launched', 'pledged', 'state', 'backers', 'usd pledged'], axis=1, inplace=True)
ks.head()

,main_category,country,usd_pledged_real,usd_goal_real,outcome
0,Publishing,GB,0.0,1533.95,0
1,Film & Video,US,2421.0,30000.00,0
2,Film & Video,US,220.0,45000.00,0
3,Music,US,1.0,5000.00,0
4,Film & Video,US,1283.0,19500.00,0


In [157]:
print(ks.main_category.unique().shape[0])
print(ks.country.unique().shape[0])

15
22


In [159]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(sparse=False)

categoricals = ['main_category', 'country']
categoricals2enc = enc.fit_transform(ks[categoricals])

In [166]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

numericals = ['usd_pledged_real', 'usd_goal_real']
numericals2sca = scaler.fit_transform(ks[numericals])

In [167]:
cols = [item for sublist in enc.categories_ for item in sublist]
#sparsecats = pd.DataFrame.sparse.from_spmatrix(categoricals2enc, columns=cols)
sparse_df = pd.DataFrame(categoricals2enc, columns=cols)
nums_df = pd.DataFrame(numericals2sca, columns=numericals)
data = ks[['outcome']].join(sparse_df).join(nums_df)

# I don't why after this last line, the resulting dataset will have +6k nans.
# Will drop for now
data.dropna(inplace=True)

In [168]:
from sklearn.model_selection import train_test_split

columns = set(data.columns)
columns.discard('outcome')

x_train, x_test, y_train, y_test = train_test_split(data[columns], data['outcome'], test_size=0.1, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=x_test.shape[0], random_state=42)

In [169]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

clf = LinearDiscriminantAnalysis()
clf.fit(x_train, y_train)

/home/lativ/.virtualenvs/dl/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)

In [171]:
from sklearn import metrics

y_pred = clf.predict(x_val)
score = metrics.roc_auc_score(y_val, y_pred)

print(f'Test AUC score: {score}')

Test AUC score: 0.5000379737221843


## LDA

## PCA